In [1]:
# Setting up
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive,FloatSlider,IntSlider
import deepdish.io as dd
import numpy as np
%matplotlib inline
def find_closest(A, target):
    #A must be sorted
    idx = A.searchsorted(target)
    idx = np.clip(idx, 1, len(A)-1)
    left = A[idx-1]
    right = A[idx]
    idx -= target - left < right - target
    return idx
path = 'results/May13-2019/'

In [2]:
chi0 = dd.load('results/May13-2019/bwh_set3_drought_dy20_dp0_05_chi0_uniform.hdf5')
chi1 = dd.load('results/May13-2019/bwh_set3_drought_dy20_dp0_05_chi1_uniform.hdf5')

In [10]:
print(chi1.keys())
print(chi1['l'])
Ps = chi0['Ps_dimensional']
print(Ps)
lattice_length=(chi1['l']/np.sqrt(Ps['M0']/Ps['DB']))[0]
print(lattice_length)
del_to = 0.3
K_min   = (1.0-del_to)
K_max   = (1.0+del_to)
K_to = lambda chi: K_max + chi*(K_min-K_max)
print(K_to(0),K_to(1))
conv_P = 1.0/((Ps['Lambda_max'])/(Ps['M0']**2))
print(conv_P)


dict_keys(['Ps_dimensional', 'T', 'b', 'h', 'l', 'n', 'p', 'w', 'chi'])
(256.0, 256.0)
{'A': 120.0, 'DB': 0.1, 'DH': 4.0, 'DW': 2.5, 'E': 1.75, 'F': 0.01, 'Gamma': 14.0, 'K': 0.5, 'Lambda_max': 0.03, 'Lambda_min': 0.025, 'M0': 2.0, 'NH': 4.5, 'NW': 1.5, 'P': 100.0, 'Q': 2.0, 'RH': 0.8, 'RW': 0.3, 'Z': 1000.0, 'del_to': 0.3, 'mu_s_max': 0.3, 'nZ': 300.0, 'npor': 0.3, 's_fc': 0.5333333333333333, 's_fos': 0.175, 's_wp': 0.085, 'seasonality': 0.0}
57.243340223994615
1.3 0.7
133.33333333333334


In [15]:
def plotB(p,save):
    fig,ax=plt.subplots(1,2,sharey=True,figsize=(2*4.5,4))
    fig.subplots_adjust(right=0.8)
    p_chi0_0 = np.amin(chi0['p'])+p*(np.amax(chi0['p'])-np.amin(chi0['p']))
    p_chi1_0 = np.amin(chi1['p'])+p*(np.amax(chi1['p'])-np.amin(chi1['p']))
    #print p
    idx_chi0_0 = len(chi0['p'])-find_closest(np.sort(chi0['p']),p_chi0_0)-1
    idx_chi1_0 = len(chi1['p'])-find_closest(np.sort(chi1['p']),p_chi1_0)-1
    #print idx
    im1 = ax[0].imshow(chi0['b'][idx_chi0_0]*K_to(0),extent=(0,lattice_length,0,lattice_length),
                       cmap=plt.cm.YlGn,vmax=max_K,vmin=0.0)
    ax[0].set_title(r'$\chi=0,\langle B \rangle = {:04.2f}\,kg/m^2$'.format(np.mean(chi0['b'][idx_chi0_0])*K_to(0)))
    im2 = ax[1].imshow(chi1['b'][idx_chi1_0]*K_to(1),extent=(0,lattice_length,0,lattice_length),
                       cmap=plt.cm.YlGn,vmax=max_K,vmin=0.0)
    ax[1].set_title(r'$\chi=0,\langle B \rangle = {:04.2f}\,kg/m^2$'.format(np.mean(chi1['b'][idx_chi1_0])*K_to(1)))
    ax[0].set_xlabel(r'$X\,[m]$')
    ax[1].set_xlabel(r'$X\,[m]$')
    ax[0].set_xlabel(r'$Y\,[m]$')
    ax[1].set_xlabel(r'$Y\,[m]$')
    cbar_ax1 = fig.add_axes([0.82, 0.3, 0.03, 0.45])
    fig.colorbar(im2, cax=cbar_ax1)
    #plt.colorbar(im1, ax=ax[0])
    #plt.tight_layout()
    print("p=",chi0['p'][idx_chi0_0],", for chi=0 <b>=",np.mean(chi0['b'][idx_chi0_0]), end=' ')
    print(",for chi=1 <b>=",np.mean(chi1['b'][idx_chi1_0]))
    if save:
        fname = path+"/bwh_drought_P{:3d}mm".format(int(chi0['p'][idx_chi0_0]*conv_P)).replace(".","_")
        print("Saved in:",fname)
        plt.savefig(fname+".png")
        plt.savefig(fname+".pdf")

In [16]:
%matplotlib notebook
p_w     = FloatSlider(min=0, max=1, step=0.01, value=1.)
save_w  = IntSlider(min=0, max=1, value=0)
interact(plotB,p=p_w,save=save_w)

interactive(children=(FloatSlider(value=1.0, description='p', max=1.0, step=0.01), IntSlider(value=0, descript…

<function __main__.plotB(p, save)>